In [4]:
from finn.core.modelwrapper import ModelWrapper
from finn.analysis.fpgadataflow.res_estimation import res_estimation

#model = ModelWrapper('intermediate_models/kernel_partitions/partition_2.onnx')
#model = ModelWrapper('model_modified.onnx')
#model = ModelWrapper('cnv_w1a1_u250.onnx')
#model = ModelWrapper('cnv_w1a1_factor_0_9.onnx')
model = ModelWrapper('partition_2.onnx')

resources = res_estimation(model)

In [5]:
from finn.util.platforms import platforms

board = "Pynq-Z1"
#board = "ZCU104"
#board = "U250"

board_resources = platforms[board]().compute_resources
max_luts = sum(res[0] for res in board_resources)

print("Max luts: {}".format(max_luts))
print("70%: {}".format(max_luts*0.7))

Max luts: 53200
70%: 37240.0


In [6]:
import numpy as np

lut_count = 0
for k,v in resources.items():
    lut_node = v.get("LUT")
    lut_count = lut_count + lut_node

print("LUT count: {}".format(lut_count))

fraction = np.round(lut_count/max_luts*100,2)
print("Fraction: {}%".format(fraction))




LUT count: 30179
Fraction: 56.73%


In [11]:
from finn.custom_op.registry import getCustomOp

model = ModelWrapper('intermediate_models/kernel_partitions/partition_2.onnx')

max_cycles=0
for n in model.graph.node:
    inst = getCustomOp(n)
    cycles = inst.get_nodeattr('cycles_estimate')
    if cycles>max_cycles:
        max_cycles=cycles
        print(cycles)
        largest_node = n

3072
24588
28224


In [81]:
print(largest_node)

input: "StreamingFIFO_3_out0"
input: "StreamingFCLayer_Batch_1_param0"
input: "StreamingFCLayer_Batch_1_param1"
output: "StreamingFCLayer_Batch_1_out0"
name: "StreamingDataflowPartition_1_StreamingFCLayer_Batch_1"
op_type: "StreamingFCLayer_Batch"
attribute {
  name: "ActVal"
  i: 0
  type: INT
}
attribute {
  name: "MH"
  i: 64
  type: INT
}
attribute {
  name: "MW"
  i: 576
  type: INT
}
attribute {
  name: "PE"
  i: 32
  type: INT
}
attribute {
  name: "SIMD"
  i: 32
  type: INT
}
attribute {
  name: "backend"
  s: "fpgadataflow"
  type: STRING
}
attribute {
  name: "binaryXnorMode"
  i: 1
  type: INT
}
attribute {
  name: "inputDataType"
  s: "BINARY"
  type: STRING
}
attribute {
  name: "mem_mode"
  s: "decoupled"
  type: STRING
}
attribute {
  name: "noActivation"
  i: 0
  type: INT
}
attribute {
  name: "numInputVectors"
  ints: 1
  ints: 28
  ints: 28
  type: INTS
}
attribute {
  name: "outputDataType"
  s: "BINARY"
  type: STRING
}
attribute {
  name: "weightDataType"
  s: "BI

In [85]:
temp_node = largest_node

temp_inst = getCustomOp(temp_node)

print(temp_inst.get_nodeattr(""))
luts = temp_inst.lut_estimation()

print(luts)

8255
